In [187]:
# Import all of the relevant dependencies to evaluate weather data and generate 
# graph outputs

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import scipy.stats as sts
import gmaps
import pprint as pp

import json
import requests
from config import apikey as g_key



In [188]:
#Importing data from WeatherPy

weather_filepath = "../WeatherPy/output_data/cities.csv"
weather_df = pd.read_csv(weather_filepath)
weather_df

,City,Latitude,Longitude,Maxium Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date,Hemisphere
0,St. George's,12.0564,-61.7485,82.40,69,20,19.57,GD,1619475978,Northern Hemisphere
1,Fengzhen,40.4394,113.1536,39.20,24,8,8.10,CN,1619475987,Northern Hemisphere
2,Pathein,16.7833,94.7333,78.08,81,63,4.81,MM,1619475974,Northern Hemisphere
3,Mataura,-46.1927,168.8643,44.01,85,98,3.00,NZ,1619475987,Southern Hemisphere
4,Butaritari,3.0707,172.7902,83.32,75,90,12.93,KI,1619475988,Northern Hemisphere
...,...,...,...,...,...,...,...,...,...,...
570,Illapel,-31.6308,-71.1653,69.75,56,0,4.65,CL,1619475971,Southern Hemisphere
571,Hofn,64.2539,-15.2082,38.91,73,16,4.16,IS,1619476174,Northern Hemisphere
572,Baruun-Urt,46.6806,113.2792,31.37,51,20,18.75,MN,1619476174,Northern Hemisphere
573,Petrolina de Goiás,-16.0950,-49.3381,80.60,73,40,1.99,BR,1619476175,Southern Hemisphere


In [189]:
#Configure gmaps to have a heatmap dictated by humidity in a given area. The maximum intensity will be dictated by the 
#maximum humidity in the dataset

gmaps.configure(api_key=g_key)

locations = weather_df[["Latitude", "Longitude"]].astype(float)


fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weather_df["Humidity (%)"], 
                              dissipating=False, 
                              max_intensity=weather_df["Humidity (%)"].max().astype(float), 
                              point_radius = 2)
    
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [190]:
#Setting up the metrics for an ideal vacation spot

vacation_df = pd.DataFrame(weather_df.loc[(weather_df["Maxium Temperature (F)"] < 80) & (weather_df["Maxium Temperature (F)"] >= 70)
                                         & (weather_df["Wind Speed (mph)"] < 10) & (weather_df["Cloudiness (%)"] == 0)])

#Creating a new dataframe according to vacation metrics
vacation_df = vacation_df.reset_index(drop=True)
vacation_df

,City,Latitude,Longitude,Maxium Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date,Hemisphere
0,Ţahţā,26.7693,31.5021,71.60,35,0,6.91,EG,1619476041,Northern Hemisphere
1,Karratha,-20.7377,116.8463,74.28,36,0,8.77,AU,1619476070,Southern Hemisphere
2,Mīnāb,27.1467,57.0801,75.20,64,0,6.91,IR,1619476088,Northern Hemisphere


In [191]:
#Identifying hotel locations within 5000 meters of the vacation locations specified above

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
hotel_dict = {"Hotel Name": [], "Rating": [],
              "Address": [],"Latitude": [], "Longitude": [], 
             "City":[], "Country":[]}
hotel_df = pd.DataFrame(hotel_dict)


# Filling a new hotel data frame with information about the hotel that is at max 5000 meters away per location 
#in the vacation df via a for loop and iterrows()
for index, row in vacation_df.iterrows():

    params = {
    
    "location": f"{vacation_df.iloc[index, 1]}, {vacation_df.iloc[index, 2]}",  
    "radius": "5000",
    "type": "lodging",
    "key": g_key,
        
}

    print(f"Location is: {params['location']}")


    # assemble url and make API request
    print(f"Retrieving hotel results for {vacation_df.iloc[index, 0]}: ")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    #try/except in case the information requested is not avaialable
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'Rating'] = results[0]['rating']
        hotel_df.loc[index, 'Latitude'] = results[0]['geometry']['location']['lat']
        hotel_df.loc[index, 'Longitude'] = results[0]['geometry']['location']['lng']
        
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Location is: 26.7693, 31.5021
Retrieving hotel results for Ţahţā: 
Closest hotel is El Salam Hotel.
------------
Location is: -20.7377, 116.8463
Retrieving hotel results for Karratha: 
Closest hotel is ibis Styles Karratha.
------------
Location is: 27.1467, 57.0801
Retrieving hotel results for Mīnāb: 
Closest hotel is Bam campground.
------------


In [192]:
# Iterating through the list of hotel infomration to extract address, city, and country of the hotel identified

base_url = "https://maps.googleapis.com/maps/api/geocode/json?"
    
for index, row in hotel_df.iterrows():

    params = {
    
    "latlng": f"{vacation_df.iloc[index, 1]},{vacation_df.iloc[index, 2]}", 
    "key": g_key,
        
}

    print(f"Location is: {params['latlng']}")


    # assemble url and make API request
    print(f"Retrieving locality and country for {hotel_df.iloc[index, 0]}: ")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        
         hotel_df.loc[index, 'Address'] = results[0]['formatted_address']
        
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        



Location is: 26.7693,31.5021
Retrieving locality and country for El Salam Hotel: 
Location is: -20.7377,116.8463
Retrieving locality and country for ibis Styles Karratha: 
Location is: 27.1467,57.0801
Retrieving locality and country for Bam campground: 


In [193]:
# Appending the previous hotel dataframe to be complete with city and country information 
marker_locations = hotel_df[['Latitude', 'Longitude']]

for index, row in hotel_df.iterrows():

    teststr = hotel_df.iloc[index,2]
    split_teststr = teststr.split(",")
    length = len(split_teststr)

    hotel_df.loc[index, 'City'] = split_teststr[length-2]
    hotel_df.loc[index, 'Country'] = split_teststr[length-1]

hotel_df

,Hotel Name,Rating,Address,Latitude,Longitude,City,Country
0,El Salam Hotel,3.4,"Unnamed Road، السوالم، مركز طهطا، سوهاج،،, الس...",26.768816,31.504883,Unnamed Road، السوالم، مركز طهطا، سوهاج،،,السوالم، طهطا، سوهاج، Egypt
1,ibis Styles Karratha,3.4,"19 Welcome Rd, Karratha WA 6714, Australia",-20.732225,116.846744,Karratha WA 6714,Australia
2,Bam campground,4.0,"Hormozgan Province, Minab, Imam Khomeini Blvd....",27.146494,57.084105,Imam Khomeini Blvd.,Iran


In [194]:
#Creating a HTML template that contains the address, city, and country information to display for each marker/hotel 
#Adding a marker per hotel identified to the heat map generated previously
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>Address</dt><dd>{Address}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig



Figure(layout=FigureLayout(height='420px'))